<a href="https://colab.research.google.com/github/Ha-eunYu/OTTO/blob/main/OTTO_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jsonlines
import jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importing Libraries
import polars as pl
from io import StringIO
import pandas as pd
import json
import os
import numpy as np
import csv

In [ ]:
# 내 계정 API Token(json) 다운 받아서 업로드
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"haeunu","key":"29e3cb0150822d980061ca25d6403725"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
# Kaggle API를 사용하기 위해서 json파일 ~/.kaggle로 이동
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

#Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# kaggle 데이터셋 API 주소 복사
# !kaggle datasets download -d theblackmamba31/landscape-image-colorization
!kaggle competitions download -c otto-recommender-system

100% 1.93G/1.94G [00:18<00:00, 165MB/s]
100% 1.94G/1.94G [00:18<00:00, 115MB/s]


In [ ]:
!ls

kaggle.json  otto-recommender-system.zip  sample_data


In [ ]:
!pwd

/content


In [ ]:
!unzip /content/otto-recommender-system.zip

Archive:  /content/otto-recommender-system.zip
  inflating: sample_submission.csv   
  inflating: test.jsonl              
  inflating: train.jsonl             


In [ ]:
with open('test.jsonl', encoding='utf-8') as file:
  test = [json.loads(line) for line in file]
test=pd.DataFrame(test)

In [ ]:
test.head()

,session,events
0,12899779,"[{'aid': 59625, 'ts': 1661724000278, 'type': '..."
1,12899780,"[{'aid': 1142000, 'ts': 1661724000378, 'type':..."
2,12899781,"[{'aid': 141736, 'ts': 1661724000559, 'type': ..."
3,12899782,"[{'aid': 1669402, 'ts': 1661724000568, 'type':..."
4,12899783,"[{'aid': 255297, 'ts': 1661724000572, 'type': ..."


This Notebook is simple demonsration of Polars Library to convert JSONL file data with around 12GB data to 1.5GB parquest data in kaggle cpu.

In [ ]:
#Install Polars with all optional dependencies.
!pip install 'polars[all]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 75.4 MB/s eta 0:00:00


In [ ]:
#checking the number of session data avilable in training data
with open('test.jsonl', encoding='utf-8') as json_file:
    file_lines = json_file.readlines()

print(f'Test data consists of {len(file_lines)} sessions')

Training data consists of 1671803 sessions


In [ ]:
1671803/12899779

0.12959935205091497

In [ ]:
#creating a polar dataframe
slice_size = 1_000_000
df = pl.concat(
    [
        pl.read_json(
            StringIO("".join(file_lines[offset: (offset + slice_size)])),
            json_lines=True,
        )
        for offset in range(0, len(file_lines), slice_size)
    ]
)
#converting struct of events in 3 columns(aid,ts,type) 
df_final=df.explode('events').unnest('events')

In [ ]:
df_final

session,aid,ts,type
i64,i64,i64,str
12899779,59625,1661724000278,"""clicks"""
12899780,1142000,1661724000378,"""clicks"""
12899780,582732,1661724058352,"""clicks"""
12899780,973453,1661724109199,"""clicks"""
12899780,736515,1661724136868,"""clicks"""
12899780,1142000,1661724155248,"""clicks"""
12899781,141736,1661724000559,"""clicks"""
12899781,199008,1661724022851,"""clicks"""
12899781,57315,1661724170835,"""clicks"""


In [ ]:
df_final.select(pl.col("type").unique())

type
str
"""clicks"""
"""orders"""
"""carts"""


In [ ]:
from_ = ["clicks","orders","carts"]
to_ = [0,1,2]


def replace(column, from_, to_):
    # initiate the expression with `pl.when`
    branch =  pl.when(pl.col(column) == from_[0]).then(to_[0])

    
    # for every value add a `when.then`
    for (from_value, to_value) in zip(from_, to_):
        branch = branch.when(pl.col(column) == from_value).then(to_value)

    # finish with an `otherwise`
    return branch.otherwise(pl.col(column)).alias(column)
    


df_final=df_final.with_column(replace("type", from_, to_))

In [ ]:
df_final

session,aid,ts,type
i64,i64,i64,str
12899779,59625,1661724000278,"""0"""
12899780,1142000,1661724000378,"""0"""
12899780,582732,1661724058352,"""0"""
12899780,973453,1661724109199,"""0"""
12899780,736515,1661724136868,"""0"""
12899780,1142000,1661724155248,"""0"""
12899781,141736,1661724000559,"""0"""
12899781,199008,1661724022851,"""0"""
12899781,57315,1661724170835,"""0"""


In [ ]:
df_final=df_final.with_column(pl.col("type").cast(pl.Int32, strict=False).alias("type"))
df_final

session,aid,ts,type
i64,i64,i64,i32
12899779,59625,1661724000278,0
12899780,1142000,1661724000378,0
12899780,582732,1661724058352,0
12899780,973453,1661724109199,0
12899780,736515,1661724136868,0
12899780,1142000,1661724155248,0
12899781,141736,1661724000559,0
12899781,199008,1661724022851,0
12899781,57315,1661724170835,0


In [ ]:
#checking the number of session data avilable in training data
with open('train.jsonl', encoding='utf-8') as json_file:
    file_lines = json_file.readlines()

print(f'Training data consists of {len(file_lines)} sessions')

In [ ]:
#creating a polar dataframe
slice_size = 1_000_000
df = pl.concat(
    [
        pl.read_json(
            StringIO("".join(file_lines[offset: (offset + slice_size)])),
            json_lines=True,
        )
        for offset in range(0, len(file_lines), slice_size)
    ]
)
#converting struct of events in 3 columns(aid,ts,type) 
df_final=df.explode('events').unnest('events')